# Table of contents
1. [Requirements](#Requirements)
2. [Introduction](#Introduction)
3. [Imports](#Imports)
    1. [Libraries](#Libraries)
    2. [Data](#Data)
4. [Data Exploration](#data-exploration)
5. [Modelling](#modelling)
    1. [Baseline](#baseline)
    2. [LSTM](#lstm)
6. [Results Analysis](#results-analysis)

# Requirements

In [ ]:
!pip install kerastuner

## Introduction

## Imports

### Libraries


In [ ]:
import sympy
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
import numpy as np
import kerastuner as kt
from kerastuner.engine.hyperparameters import HyperParameters

### Data

In [ ]:
# Load the data

# Split the data
X_train, y_train, X_val, y_val = load_data(hourly_energy_consumption, hourly_temperature, hourly_radiation)


## Data Exploration

## Modelling

In [ ]:
# Aqui tratar de qualquer operação ou criação de variáveis que sejam
# necessárias para o processo de modelação de DL.
# Number of samples
n_samples = len(hourly_energy_consumption)

# Number of time steps
n_timesteps = 24
# Number of features
n_features = 3
# Reshape the data into a 3D array to feed the Neural Netwokrs
X = np.empty((n_samples, n_timesteps, n_features))
X[:, :, 0] = hourly_energy_consumption
X[:, :, 1] = hourly_temperature
X[:, :, 2] = hourly_radiation

### Baseline
        Persistence

A persistência é o método de baseline mais condiserado e que, como o nome indica, considera que o valor para o futuro é igual à ultima observação. Pode ser denotado pela seguinte equação:

$ T_{t+1} = T_{t} $ 

In [ ]:
# creating the persistence matrix
# df.merged.shape should be Amount of predictions we want to gather X 24
persistence_forecasts=np.zeros((dfmerged.shape),dtype=float)
for i in range(len(dfmerged)):
    persistence_forecasts.iloc[i,:]=dfmerged.iloc[i,-1]

### LSTM

In [ ]:
# Define the model
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(24))
    
    # Choose an optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'sgd','rmsprop'])
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    elif optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    else:    
        optimizer = tf.keras.optimizers.RMSprop(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
        
    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model
# Define the search space for Keras Tuner
hps = HyperParameters()
hps.Choice('batch_size', [32, 64, 128, 256])
hps.Choice('activation', ['relu', 'tanh','sigmoid'])
# Use the Keras Tuner to search for the best set of hyperparameters
tuner = kt.Hyperband(build_model, hps)
tuner.search(X_train, y_train, epochs=100,batch_size=1, validation_data=(X_val, y_val))
# Get the best model from the search
best_model = tuner.get_best_model()
# Use the best model to make predictions on the test set
y_pred = best_model.predict(X_test)


## Results Analysis